In [1]:
import jax
from typing import Any, Callable, Sequence, Optional
from jax import lax, random, numpy as jnp
import flax
from flax import linen as nn
import sympy as sy
import numpy as np
import sys
sys.path.append("..")
from eql.eqlearner import EQL
from eql.symbolic import get_symbolic_expr, get_symbolic_expr_layer
from typing import List, Tuple, Callable
from functools import partial
import matplotlib.pyplot as plt
import scipy
sys.path.append("../../orient")
import optax

from np_utils import flatten, unflatten

In [2]:
funs = ['mul', 'cos', 'sin']*2
e = EQL(n_layers=2, functions=funs, features=1)
key = random.PRNGKey(0)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
N = 1000
xdim = 1
x = (random.uniform(key, (N, xdim))-.5) * 2

#y = x[:,0] + jnp.cos(x[:,1])
y = jnp.cos(x) + 1 - x**2 

In [4]:
params = e.init({'params':key}, x);

In [5]:
params

FrozenDict({
    params: {
        layers_0: {
            linear_layer: {
                kernel: Array([[ 0.6292481 , -0.54229003, -0.4677973 ,  0.70746535,  0.09610943,
                         0.4903576 ,  0.06954016,  0.31511047]], dtype=float32),
                bias: Array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
            },
        },
        layers_1: {
            linear_layer: {
                kernel: Array([[ 0.19629148,  0.06706532,  0.00969497,  0.23161124, -0.11875771,
                         0.01930422,  0.05764793, -0.0596979 ],
                       [ 0.4696742 ,  0.46638718,  0.21155392, -0.8553514 ,  0.6172892 ,
                        -0.44233188,  0.22190586, -0.18106432],
                       [-0.44605988, -0.20469512, -0.32464647,  0.5807687 ,  0.02217815,
                        -0.2559955 ,  0.17005439,  0.71249163],
                       [ 0.45161504, -0.10305008, -0.47209048,  0.35264972, -0.3844344 ,
                         0.2527192 , -0

In [6]:
def mse_fn(params):
    pred = e.apply(params, x)
    return jnp.mean((pred-y)**2)

In [7]:
def loss(params):
    return mse_fn(params)
loss_grad_fn = jax.jit(jax.value_and_grad(loss))

In [8]:
tx = optax.adam(learning_rate=1e-2)
opt_state = tx.init(params)
loss_grad_fn = jax.jit(jax.value_and_grad(loss))

In [9]:
for i in range(10000):
    loss_val, grads = loss_grad_fn(params)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 99 == 0:
        print(loss_val)

2.2196107
0.0027071456
0.00042087326
0.0002654106
0.00011064316
2.0543643e-05
4.4613835e-06
3.6789131e-06
3.520719e-06
3.36399e-06
3.2019113e-06
3.0358647e-06
2.8672011e-06
2.6971727e-06
2.5271006e-06
2.3581579e-06
2.1915357e-06
2.0283371e-06
1.8695941e-06
1.7162446e-06
1.5692009e-06
1.4292251e-06
1.2970004e-06
1.1730463e-06
1.0577919e-06
9.515116e-07
8.54322e-07
7.6624815e-07
6.871244e-07
6.1663485e-07
8.6005423e-07
4.960215e-07
4.5028426e-07
5.873193e-07
3.7753443e-07
3.5102744e-07
5.0225583e-07
3.1070547e-07
3.4219494e-07
2.9524352e-07
2.7227787e-07
1.4357581e-05
2.6714446e-07
2.4733802e-07
2.2175887e-05
2.4136529e-07
2.2805808e-07
5.8675296e-06
2.2211954e-07
2.1000004e-07
2.6261935e-07
2.0675199e-07
2.0703268e-07
2.1314641e-07
1.9277927e-07
5.2044e-07
2.098606e-07
1.8224698e-07
1.7019016e-07
3.334164e-07
1.7446423e-07
1.6227581e-07
1.8598286e-06
1.643442e-07
1.522236e-07
7.825269e-07


KeyboardInterrupt: 

In [10]:
params

FrozenDict({
    params: {
        last: {
            bias: Array([0.2339758], dtype=float32),
            kernel: Array([[ 0.58265454],
                   [ 0.9746547 ],
                   [-0.45646206],
                   [ 0.17972724],
                   [ 0.04261871],
                   [-0.43190786]], dtype=float32),
        },
        layers_0: {
            linear_layer: {
                bias: Array([-0.04613144,  0.04685027, -0.30019155, -0.1410033 , -0.12093793,
                       -0.07968189, -0.38642558, -0.24723455], dtype=float32),
                kernel: Array([[ 0.9059607 , -0.829572  , -0.6477162 ,  0.73437506, -0.0984896 ,
                         0.33892304, -0.6802242 ,  0.03162879]], dtype=float32),
            },
        },
        layers_1: {
            linear_layer: {
                bias: Array([-0.05229618, -0.04547407,  0.12206765,  0.20241621,  0.43422917,
                       -0.07817697,  0.24973992, -0.021704  ], dtype=float32),
                ke

In [11]:
symb = get_symbolic_expr(params, funs)[0]
print(symb)

-0.43190786242485*(-1.26319241523743*(0.0468502715229988 - 0.829572021961212*x0)*(0.90596067905426*x0 - 0.0461314432322979) + 0.0679193958640099*(-0.0984895974397659*x0 - 0.120937928557396)*(0.338923037052155*x0 - 0.0796818882226944) + 0.333707571029663*sin(0.0316287912428379*x0 - 0.247234553098679) - 0.0860139578580856*sin(0.734375059604645*x0 - 0.141003295779228) - 0.173952579498291*cos(0.647716224193573*x0 + 0.300191551446915) - 0.439607203006744*cos(0.680224180221558*x0 + 0.386425584554672) - 0.0781769677996635)*(-0.383745849132538*(0.0468502715229988 - 0.829572021961212*x0)*(0.90596067905426*x0 - 0.0461314432322979) - 0.27533170580864*(-0.0984895974397659*x0 - 0.120937928557396)*(0.338923037052155*x0 - 0.0796818882226944) - 0.667306184768677*sin(0.0316287912428379*x0 - 0.247234553098679) + 0.414273023605347*sin(0.734375059604645*x0 - 0.141003295779228) + 0.267243564128876*cos(0.647716224193573*x0 + 0.300191551446915) + 0.379908174276352*cos(0.680224180221558*x0 + 0.386425584554672